In [1]:
import os
# Tell theano to use our GPU
os.environ['THEANO_FLAGS'] = "device=gpu0"

import keras
import numpy as np

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 1080 (CNMeM is disabled, cuDNN 5110)


# TODO: Some intro stuff here maybe?

# Cats vs Dogs

Note: This slightly follows lesson 1 from [course.fast.ai](http://course.fast.ai/lessons/lesson1.html).

We're going to use Keras to participate in the [Dogs vs Cats Redux Kaggle competition](https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition) while exploring Keras along the way.



The goal of this competition is to predict whether a given image is of a cat or a dog. 

## Set up the data

[Kaggle gives us](https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition/data) two folders: train and test.

* The train folder is comprised of 12,500 cats and 12,500 dogs. These cat images are labelled cat.0.jpg through cat.12499.jpg. The opposite is true for dogs.

* The test folder has 12,500 unlabelled images.


We will start by creating our testing, training, and validation set. We will use 30% of the training images for validation.

I will assume our train folder is located at _data/train_.

### Get the list of the train / test files

In [25]:
# Since we have no validation set, we need to split our existing training set into train/valid
# We will use this amount of the old training set for our new training set. The rest will be for valid.
percent_train = 0.7 
path = 'data'

train_path = path + '/train'
valid_path  = path + '/valid'

# Get all of the files in the directory
train_files = os.listdir(train_path)

# Make a list of only the cats and dogs
cats = [s for s in train_files if s[:3] == 'cat']
dogs = [s for s in train_files if s[:3] == 'dog']

# Sanity Check
print('Cats:\n', cats[:10], '\nAmount:\n', len(cats))
print('Dogs:\n', dogs[:10], '\nAmount:\n', len(dogs))

train_cat_amt = int(percent_train * len(cats))
train_dog_amt = int(percent_train * len(dogs))

cats_train = cats[:train_cat_amt]
cats_valid = cats[train_cat_amt:]

dogs_train = dogs[:train_dog_amt]
dogs_valid = dogs[train_dog_amt:]

print(len(cats_train),len(cats_valid),len(cats))
print(len(dogs_train),len(dogs_valid),len(dogs))

# Make sure we didn't lose any from an index error
assert len(cats_train) + len(cats_valid) == len(cats)
assert len(dogs_train) + len(dogs_valid) == len(dogs)

Cats:
 ['cat.0.jpg', 'cat.1.jpg', 'cat.10.jpg', 'cat.100.jpg', 'cat.1000.jpg', 'cat.10000.jpg', 'cat.10001.jpg', 'cat.10002.jpg', 'cat.10003.jpg', 'cat.10004.jpg'] 
Amount:
 12500
Dogs:
 ['dog.0.jpg', 'dog.1.jpg', 'dog.10.jpg', 'dog.100.jpg', 'dog.1000.jpg', 'dog.10000.jpg', 'dog.10001.jpg', 'dog.10002.jpg', 'dog.10003.jpg', 'dog.10004.jpg'] 
Amount:
 12500
8750 3750 12500
8750 3750 12500


### Put all of the cats in a cat folder and all of the dogs in a dog folder

In [26]:
# Make Folders to put them in if they don't exist
def make_dir_if_needed(path):
    if not os.path.isdir(path):
        os.mkdir(path)
        
make_dir_if_needed(train_path + '/cats')
make_dir_if_needed(train_path + '/dogs')

make_dir_if_needed(valid_path)
make_dir_if_needed(valid_path + '/cats')
make_dir_if_needed(valid_path + '/dogs')

In [27]:
from shutil import move

'''TRAIN'''

# Move the cats to the cat directory
for cat in cats_train:
    move(train_path + '/' + cat, train_path + '/cats/' + cat)

# Move the dogs to the dog directory
for dog in dogs_train:
    move(train_path + '/' + dog, train_path + '/dogs/' + dog)
    
    
'''VALID'''

for cat in cats_valid:
    move(train_path + '/' + cat, valid_path + '/cats/' + cat)
    
    
for dog in dogs_valid:
    move(train_path + '/' + dog, valid_path + '/dogs/' + dog)


We now have all of our cats in cat folders and all our dogs in dog folders. We also made our training, testing, and validation set on the filesystem.

Here is the current structure of our directory:

In [34]:
'''
data
    test
    train
        cats
        dogs
    valid
        cats
        dogs
'''

'\ndata\n    test\n    train\n        cats\n        dogs\n    valid\n        cats\n        dogs\n'

### Load the data using Keras

Now that we set up our data into a folder for each class, we may now load the data using Keras. This is easily done with an [ImageDataGenerator](https://keras.io/preprocessing/image/#imagedatagenerator). MORE INFO HERE

In [37]:
from keras.preprocessing import image

generator = image.ImageDataGenerator()

# Credit: https://github.com/fastai/courses/blob/master/deeplearning1/nbs/vgg16.py#L142
# TODO: Explain this function
def get_batches(path, shuffle = True, batch_size = 32, class_mode = 'categorical'):
    image.ImageDataGenerator().flow_from_directory(path, target_size = (224, 224), 
                           class_mode = class_mode, shuffle = shuffle, batch_size = batch_size)

train_batches = get_batches(train_path)
valid_batches = get_batches(valid_path)



Found 17500 images belonging to 2 classes.
Found 7500 images belonging to 2 classes.
